# 测试、调试和异常
---

### 测试 stdout 输出

In [1]:
def urlprint(protocol, host, domain):
    url = '{}://{}.{}'.format(protocol, host, domain)
    print(url)

In [2]:
from io import StringIO
from unittest import TestCase
from unittest.mock import patch


class TestURLPrint(TestCase):

    def test_url_gets_to_stdout(self):
        protocol = 'http'
        host = 'www'
        domain = 'example.com'
        # 期望输出
        expected_url = '{}://{}.{}\n'.format(protocol, host, domain)

        with patch('sys.stdout', new=StringIO()) as fake_out:
            urlprint(protocol, host, domain)
            self.assertEqual(fake_out.getvalue(), expected_url)

---

### 在单元测试中给对象打补丁

使用 `unittest.mock.patch` 函数，用于断言它们在测试中的期望行为（比如，断言被调用时的参数个数，访问指定的属性等）。

```python
from unittest.mock import patch
import example

# 方法一
@patch('example.func')
def test1(x, mock_func):
    example.func(x)
    mock_func.assert_called_with(x)

# 方法二
with patch('example.func') as mock_func:
    example.func(x)
    mock_func.assert_called_with(x)

# 方法三
p = patch('example.func')
mock_func = p.start()
example.func(x)
mock_func.assert_called_with(x)
p.stop()

# 多个对象
@patch('example.func1')
@patch('example.func2')
@patch('example.func3')
def test1(mock1, mock2, mock3):
    ...

def test2():
    with patch('example.patch1') as mock1, \
         patch('example.patch2') as mock2, \
         patch('example.patch3') as mock3:
    ...
```

---

### 在单元测试中测试异常情况

测试抛出的异常

In [3]:
import unittest

def parse_int(s):
    return int(s)


class TestConversion(unittest.TestCase):

    def test_bad_int(self):
        self.assertRaises(ValueError, parse_int, 'N/A')

测试异常的具体值

In [4]:
import errno


class TestIO(unittest.TestCase):

    def test_file_not_found(self):
        try:
            f = open('/file/not/found')
        except IOError as e:
            self.assertEqual(e.errno, errno.ENOENT)
        else:
            self.fail('IOError not raised')

同时测试异常和异常值

In [5]:
class TestConversion(unittest.TestCase):

    def test_bad_int(self):
        self.assertRaisesRegex(
            ValueError, 'invalid literal .*', parse_int, 'N/A')

---

### 将测试输出用日志记录到文件中

保存如下代码后，测试文件就是可执行的，并且会将运行测试的结果打印到标准输出上

```python
import unittest

class MyTest(unittest.TestCase):
    pass

if __name__ == '__main__':
    unittest.main()
```

如果想重定向输出，就需要像下面这样修改 `main` 函数：

```python
import sys

def main(out=sys.stderr, verbosity=2):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromModule(sys.modules[__name__])
    unittest.TextTestRunner(out,verbosity=verbosity).run(suite)

if __name__ == '__main__':
    with open('testing.out', 'w') as f:
        main(f)
```

---

### 忽略或期望测试失败

```python
import unittest
import os
import platform

class Tests(unittest.TestCase):
    def test_0(self):
        self.assertTrue(True)

    @unittest.skip('skipped test')
    def test_1(self):
        self.fail('should have failed!')

    @unittest.skipIf(os.name=='posix', 'Not supported on Unix')
    def test_2(self):
        import winreg

    @unittest.skipUnless(platform.system() == 'Darwin', 'Mac specific test')
    def test_3(self):
        self.assertTrue(True)

    @unittest.expectedFailure
    def test_4(self):
        self.assertEqual(2+2, 5)

if __name__ == '__main__':
    unittest.main()
```

将如上代码保存后，在 Mac 系统的命令行执行
```
$ python3 testsample.py -v
test_0 (__main__.Tests) ... ok
test_1 (__main__.Tests) ... skipped 'skipped test'
test_2 (__main__.Tests) ... skipped 'Not supported on Unix'
test_3 (__main__.Tests) ... ok
test_4 (__main__.Tests) ... expected failure

--------------------------------------------------------------
Ran 5 tests in 0.002s

OK (skipped=2, expected failures=1)
```